## LC Prompt Self consistency

In [3]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from tqdm import tqdm
import time, re

llm_main = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)

random_numbers = []
start_time = time.time()

for _ in tqdm(range(100), desc="Sequential Requests"):
    prompt = HumanMessage(content="Give me a random number between 1 and 100.")
    response = llm_main.invoke([prompt])
    random_numbers.append(response.content)

end_time = time.time()
print(f"Time taken: {end_time - start_time:.2f} seconds")

# Extract numbers
numbers = [int(m.group()) for r in random_numbers if (m := re.search(r'\d+', r))]
print("Extracted Numbers:", numbers)


Sequential Requests:   0%|          | 0/100 [00:00<?, ?it/s]

Sequential Requests: 100%|██████████| 100/100 [00:10<00:00,  9.10it/s]

Time taken: 11.00 seconds
Extracted Numbers: [53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53]


In [4]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time, re

llm_main = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)

def ask_random_number():
    prompt = HumanMessage(content="Give me a random number between 1 and 100.")
    try:
        response = llm_main.invoke([prompt])
        return response.content
    except Exception as e:
        return f"Error: {e}"

NUM_REQUESTS = 100
responses = []

start_time = time.time()
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(ask_random_number) for _ in range(NUM_REQUESTS)]
    for f in tqdm(as_completed(futures), total=NUM_REQUESTS, desc="Parallel Requests"):
        responses.append(f.result())
end_time = time.time()

print(f"Time taken: {end_time - start_time:.2f} seconds")

# Extract numbers
numbers = [int(m.group()) for r in responses if (m := re.search(r'\d+', r))]
print("Extracted Numbers:", numbers)


Parallel Requests:   0%|          | 0/100 [00:00<?, ?it/s]

Parallel Requests: 100%|██████████| 100/100 [00:06<00:00, 14.46it/s]

Time taken: 6.94 seconds
Extracted Numbers: [53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53]


## temperature and randomness checks

In [6]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from tqdm import tqdm
import time

# 1. Initialize model
llm_main = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,  # use 0.0 for deterministic outputs
    base_url="http://localhost:11434",
    api_key="ollama"
)

# 2. Collect dog names
dog_names = []
start_time = time.time()

for _ in tqdm(range(100), desc="Getting Dog Names"):
    prompt = HumanMessage(content="Give me a random name of a dog:")
    response = llm_main.invoke([prompt])
    dog_names.append(response.content.strip())

end_time = time.time()
print(f"\nTime taken: {end_time - start_time:.2f} seconds")

# 3. Print summary
unique_names = set(dog_names)
print(f"\nTotal names collected: {len(dog_names)}")
print(f"Unique names: {len(unique_names)}")
print("\nSample names:", list(unique_names)[:10])


Getting Dog Names:   0%|          | 0/100 [00:00<?, ?it/s]

Getting Dog Names: 100%|██████████| 100/100 [00:16<00:00,  6.09it/s]


Time taken: 16.41 seconds

Total names collected: 100
Unique names: 1

Sample names: ["The random name I've generated is... **Bramble**!"]
